In [6]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
len(text)
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [9]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hello bitch"))
print(decode(encode("Hello bitch")))

[20, 43, 50, 50, 53, 1, 40, 47, 58, 41, 46]
Hello bitch


In [11]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)

In [13]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [14]:
block_size = 8
train_data[:block_size+1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"WHen input is {context} the target is {target}")

WHen input is tensor([18]) the target is 47
WHen input is tensor([18, 47]) the target is 56
WHen input is tensor([18, 47, 56]) the target is 57
WHen input is tensor([18, 47, 56, 57]) the target is 58
WHen input is tensor([18, 47, 56, 57, 58]) the target is 1
WHen input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
WHen input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
WHen input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [16]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('---------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------
When input is [24] the target is 43
When input is [24, 43] the target is 58
When input is [24, 43, 58] the target is 5
When input is [24, 43, 58, 5] the target is 57
When input is [24, 43, 58, 5, 57] the target is 1
When input is [24, 43, 58, 5, 57, 1] the target is 46
When input is [24, 43, 58, 5, 57, 1, 46] the target is 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
When input is [44] the target is 53
When input is [44, 53] the target is 56
When input is [44, 53, 56] the target is 1
When input is [44, 53, 56, 1] the target is 58
When input is [44, 53, 56, 1, 58] the tar

In [38]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
                
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim =-1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([256, 65])
tensor(4.7405, grad_fn=<NllLossBackward0>)


In [39]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [40]:
batch_size = 32
for steps in range(10000):

    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

4.648484230041504
4.650679588317871
4.710906505584717
4.711498737335205
4.70780086517334
4.676126480102539
4.692947864532471
4.781689167022705
4.717318534851074
4.819327354431152
4.7402472496032715
4.69600248336792
4.769904136657715
4.705028533935547
4.778793811798096
4.6513991355896
4.644524574279785
4.8274664878845215
4.707894802093506
4.787680149078369
4.635769367218018
4.669399738311768
4.713061332702637
4.770639419555664
4.697976589202881
4.6887688636779785
4.733644008636475
4.644672870635986
4.661499500274658
4.652538776397705
4.7977776527404785
4.667324066162109
4.801926612854004
4.67767858505249
4.739298343658447
4.768125057220459
4.6637372970581055
4.666025161743164
4.7944793701171875
4.707888603210449
4.695466995239258
4.784860134124756
4.642155170440674
4.564848899841309
4.563507556915283
4.7707953453063965
4.626956462860107
4.638504981994629
4.716789722442627
4.6497626304626465
4.581180095672607
4.705897331237793
4.717549800872803
4.657068252563477
4.714186191558838
4.65794

In [42]:
idxes = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx = idxes, max_new_tokens = 300)[0].tolist()))


'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLengerithesinthengove fal amas trr
TI ar I t, mes, o sar; my w, fredeeyove
THek' merer, dd
We ntem lud engitonso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


In [43]:
torch.manual_seed(1337)
B,T,C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [45]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t]=torch.mean(xprev,0)

In [46]:
wei = torch.tril(torch.ones(T, T))
wei = wei/ wei.sum(1, keepdim = True)
xbow2 = wei @ x